In [7]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 40.2 MB/s eta 0:00:00


In [67]:
#Import different libraries
#Importations de Bibliothèques Standard
import os
import base64
from io import BytesIO

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import mysql.connector
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array
from tensorflow.keras.utils import to_categorical


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from google.cloud import storage


In [9]:
# Load the model from GCP bucket
BUCKET_NAME = 'pulmobucket_models'
MODEL_PATH = 'GCPmodel.pkl'
local_model_path = '/tmp/GCPmodel.pkl'

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(MODEL_PATH)
blob.download_to_filename(local_model_path)

with open(local_model_path, 'rb') as file:
    model = pickle.load(file)


In [41]:
#Define parameters
img_height, img_width = 150, 150
batch_size = 32
epochs = 30

In [15]:
#Database connection details
DB_HOST = '34.155.61.153'
DB_PORT = '3306'
DB_NAME = 'PulmoScanAI'
DB_USER = 'pulmoscanai'
DB_PASS = 'pulmo'

In [53]:
#Connect to MySQL database
conn = mysql.connector.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASS
)

#Define the SQL query
QUERY = """
SELECT image_data,
CASE
  WHEN modified = 1 THEN final_prediction
  ELSE original_prediction
END AS label
FROM user_predictions
"""

#Run the query and load data into a DataFrame
df = pd.read_sql_query(QUERY, conn)

#Close the connection
conn.close()

<ipython-input-53-08859a97bf19>:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(QUERY, conn)


In [57]:
#Decode images and resize them
def decode_and_resize_image(image_data):
    try:
        if not image_data:
            raise ValueError("Empty image data")
        image = Image.open(BytesIO(image_data)).convert('RGB')  # Convertir en RGB
        image = image.resize((img_width, img_height))  # Redimensionner les images
        return image
    except Exception as e:
        print(f"Error decoding image: {e}")
        return None

df['image'] = df['image_data'].apply(decode_and_resize_image)

#Filtrer les lignes avec des images non décodées
df = df[df['image'].notnull()]

#Convert PIL images to arrays
df['image_array'] = df['image'].apply(img_to_array)

labels = df['label'].astype('category').cat.codes
labels = to_categorical(labels, num_classes=len(df['label'].unique()))

images = np.array(df['image_array'].tolist())

In [58]:
#Prepare the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_generator = train_datagen.flow(
    images,
    labels,
    batch_size=batch_size
)

In [70]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks
callbacks = [
    EarlyStopping(monitor='loss', patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath='/tmp/cnn_model.keras')
]

In [71]:
history = model.fit(
    train_generator,
    steps_per_epoch=max(1, len(df) // batch_size),
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/30
1/1 [==============================] - 14s 14s/step - loss: 0.8269 - accuracy: 0.6667
Epoch 2/30
1/1 [==============================] - 13s 13s/step - loss: 0.8353 - accuracy: 0.6667
Epoch 3/30
1/1 [==============================] - 13s 13s/step - loss: 0.5469 - accuracy: 0.8095
Epoch 4/30
1/1 [==============================] - 14s 14s/step - loss: 0.9782 - accuracy: 0.5238
Epoch 5/30
1/1 [==============================] - 13s 13s/step - loss: 0.4336 - accuracy: 0.7619
Epoch 6/30
1/1 [==============================] - 13s 13s/step - loss: 1.0546 - accuracy: 0.5714
Epoch 7/30
1/1 [==============================] - 13s 13s/step - loss: 0.6706 - accuracy: 0.6667
Epoch 8/30
1/1 [==============================] - 13s 13s/step - loss: 0.4495 - accuracy: 0.8571


In [74]:
#Save the updated model back to the GCP bucket
local_model_path_incremental = '/tmp/cnn_model_incremental.pkl'

#Convert model to pickle
with open(local_model_path_incremental, 'wb') as file:
    pickle.dump(model, file)

#Upload the pickle model to GCP bucket
blob = bucket.blob('GCP_cnn_model_incremental.pkl')
blob.upload_from_filename(local_model_path_incremental)
